In [ ]:
options(jupyter.rich_display = FALSE)

# Basic data science with R

## Reshaping data

Multivariate data can be represented in wide or long format:

- Reshaping a data in wide format into a long format is known as "melting"
- Reshaping a data in long format into a wide format is known as "casting"

Different shapes of data can be utilized for specific purposes

### Melting

A chessboard is in a 8x8 shape

Suppose we have a chessboard with random configuration: Each of the 32 pieces by any player is given a unique number and empty squares are 0:

First create a random vector:

In [ ]:
chess_vec <- integer(64)
chess_vec[1:32] <- 1:32
chess_vec_r <- sample(chess_vec)
chess_vec_r

And convert to a matrix:

In [ ]:
chess_mat <- matrix(chess_vec_r, nrow = 8)
chess_mat

In [ ]:
chess_df <- as.data.frame(chess_mat)
chess_df

Our task is to create a function to convert this matrix into a format in which we have three columns:
- One for the row index of the original matrix
- One for the column index of the original matrix
- One for the value

We will use the base function reshape()

In [ ]:
?reshape

```
reshape(data, varying = NULL, v.names = NULL, timevar = "time",
             idvar = "id", ids = 1:NROW(data),
             times = seq_along(varying[[1]]),
             drop = NULL, direction, new.row.names = NULL,
             sep = ".",
             split = if (sep == "") {
                 list(regexp = "[A-Za-z][0-9]", include = TRUE)
             } else {
                 list(regexp = sep, include = FALSE, fixed = TRUE)}
             )
     
Arguments:

    data: a data frame

 varying: names of sets of variables in the wide format that correspond
          to single variables in long format (‘time-varying’).  This is
          canonically a list of vectors of variable names, but it can
          optionally be a matrix of names, or a single vector of names.
          In each case, the names can be replaced by indices which are
          interpreted as referring to ‘names(data)’.  See ‘Details’ for
          more details and options.

 v.names: names of variables in the long format that correspond to
          multiple variables in the wide format.  See ‘Details’.

 timevar: the variable in long format that differentiates multiple
          records from the same group or individual.  If more than one
          record matches, the first will be taken (with a warning).

   idvar: Names of one or more variables in long format that identify
          multiple records from the same group/individual.  These
          variables may also be present in wide format.

     ids: the values to use for a newly created ‘idvar’ variable in
          long format.

   times: the values to use for a newly created ‘timevar’ variable in
          long format.  See ‘Details’.

    drop: a vector of names of variables to drop before reshaping.

direction: character string, partially matched to either ‘"wide"’ to
          reshape to wide format, or ‘"long"’ to reshape to long
          format.
```

First, let's convert the matrix into a data.frame:

In [ ]:
chess_df <- as.data.frame(chess_mat)
chess_df

- idvar is the variable which need to be left unaltered
- varying are the ones that needs to converted from wide to long
- v.names are the values that should be against the times in the resultant data frame.
- new.row.names is used to assign row names to the resultant dataset
- direction is, to which format the data needs to be transformed

In [ ]:
chess_long <- reshape(chess_df,
                      idvar = "rows",
                      varying = 1:8,
                      v.name = "piece",
                      direction = "long")

In [ ]:
chess_long

We can change the default name of column "time" to "cols" and rearrange the columns: 

In [ ]:
names(chess_long)[1] <- "cols"
chess_long <- chess_long[,c("cols", "rows", "piece")]
chess_long

### Casting

Now let's convert the long chessboard back to a wide 8x8 one:

- idvar is the variable which need to be left unaltered
- timevar are the variables that needs to converted to wide format
- v.names are the value variable
- direction is, to which format the data needs to be transformed

In [ ]:
chess_wide <- reshape(chess_long,
       idvar = "rows",
       v.names = "piece",
       timevar = "cols",
       direction = "wide")

chess_wide

We delete the first column, which is unneccessary in our example:

In [ ]:
chess_wide <- chess_wide[,-1]
chess_wide

and let's compare whether all values are unchanged vis-a-vis the original object:

In [ ]:
all(chess_wide == chess_mat)

### Real data example: Reshaping IMF WEO database

First read the tsv data into R using read.delim() function (a read.table wrapper for tsv files):

In [ ]:
?read.table

In [ ]:
#weo <- read.table("data/WEO_Data.xls", sep = "\t", header = T, na.strings = c("n/a", "--", ""), stringsAsFactors = T, dec = ".")
weo <- read.delim("data/WEO_Data.xls", na.strings = c("n/a", "--", ""), stringsAsFactors = T, dec = ".")
#weo <- read.delim("~/Downloads/WEO_Data.xls", na.strings = c("n/a", "--", ""), stringsAsFactors = T, dec = ".")

Let's view the structure of the data:

In [ ]:
str(weo)

Let's get the names and indices of factor variables:

In [ ]:
factors <- which(sapply(weo, is.factor))
factors

And get the unique values (or levels) of each of these factor variables:

In [ ]:
lapply(weo[, factors], levels)

#### Melting GDP per capita (PPP)

Now we are concerned with only the NGDPRPPPPC series "Purchasing power parity; 2011 international dollar"

So we filter the dataset:

In [ ]:
weo_ppp <- weo[weo$WEO.Subject.Code == "NGDPRPPPPC",]

In [ ]:
str(weo_ppp)

Now we want a long DF of four columns: Country names, country codes, years and per capita gdp in ppp values 

In [ ]:
names(weo_ppp)

In [ ]:
cols <- sprintf("X%s", 1980:2023)
cols

In [ ]:
weo_ppp_long <- reshape(weo_ppp,
                      idvar = c("WEO.Country.Code", "Country"),
                      varying = cols,
                        times = cols,
                      v.name = "NGDPRPPPPC",
                      direction = "long")

In [ ]:
str(weo_ppp_long)

We can clear rownames:

In [ ]:
rownames(weo_ppp_long) <- NULL

In [ ]:
weo_ppp_long_2 <- weo_ppp_long[,c("Country", "WEO.Country.Code", "ISO", "time", "NGDPRPPPPC")]
weo_ppp_long_2

We can convert year values from X1980 to 1980 and make them numeric:

In [ ]:
weo_ppp_long_2$time <- as.integer(gsub("X", "", weo_ppp_long_2$time))
weo_ppp_long_2

#### Casting 2016 data

Now let's select only a single year, e.g 2016 and create a wide matrix of subjects on the columns and countries on the rows:

In [ ]:
names(weo_ppp)

In [ ]:
weo_2016 <- weo[,c(names(weo_ppp)[1:7], "X2016")]

In [ ]:
str(weo_2016)

In [ ]:
?reshape

In [ ]:
weo_2016_wide <- reshape(weo_2016,
                      idvar = c("WEO.Country.Code", "Country"),
                      v.names = "X2016",
                      timevar = "WEO.Subject.Code",
                      drop = c("Subject.Descriptor", "Units", "Scale"),
                      direction = "wide")

In [ ]:
str(weo_2016_wide)

In [ ]:
weo_2016_wide

We may have to correct column names:

In [ ]:
names_2016 <- names(weo_2016_wide)
names_2016

In [ ]:
names(weo_2016_wide) <- gsub("X2016\\.", "", names_2016)

In [ ]:
str(weo_2016_wide)

### Extension packages

reshape2, tidy2 (from tidyverse) and data.table packages provide extended functionality on reshaping datasets

## Discretize numeric data

Now we may want to create factors on GDP per capita (PPP): Low income, middle income and high income countries  

Let's say the cutting points are 3000 and 12000

In [ ]:
str(weo_ppp_long_2)

We will discretize the NGDPRPPPPC column with cut() function and add the results as a new factor variable:

In [ ]:
max(weo_ppp_long_2$NGDPRPPPPC, na.rm = T)

In [ ]:
weo_ppp_long_2$income_levels <- cut(weo_ppp_long_2$NGDPRPPPPC,
                                    breaks = c(0, 5000, 20000, max(weo_ppp_long_2$NGDPRPPPPC, na.rm = T)),
                                    labels = c("L", "M", "H"))

In [ ]:
weo_ppp_long_2[,3:6]

classInt package offers alternative methods for discretizing numeric values

## Joining data

Now let's import a second IMF dataset: 2016 values of international investment position of all countries

In [ ]:
iip <- read.csv("data/Data_in_US_Dollars.csv")

In [ ]:
str(iip)

Now similar to SQL joins, we will "merge" the weo_2016_wide and iip datasets using common "WEO.Country.Code" variable:

In [ ]:
?merge

```
merge(x, y, by = intersect(names(x), names(y)),
           by.x = by, by.y = by, all = FALSE, all.x = all, all.y = all,
           sort = TRUE, suffixes = c(".x",".y"), no.dups = TRUE,
           incomparables = NULL, ...)
     
Arguments:

    x, y: data frames, or objects to be coerced to one.

by, by.x, by.y: specifications of the columns used for merging.  See
          ‘Details’.

     all: logical; ‘all = L’ is shorthand for ‘all.x = L’ and ‘all.y =
          L’, where ‘L’ is either ‘TRUE’ or ‘FALSE’.

   all.x: logical; if ‘TRUE’, then extra rows will be added to the
          output, one for each row in ‘x’ that has no matching row in
          ‘y’.  These rows will have ‘NA’s in those columns that are
          usually filled with values from ‘y’.  The default is ‘FALSE’,
          so that only rows with data from both ‘x’ and ‘y’ are
          included in the output.

   all.y: logical; analogous to ‘all.x’.

```

In [ ]:
nrow(weo_2016_wide)
nrow(iip)

As we see iip table has more rows

Let's see the rows that are common, that only weo_2016_wide have or that only iip have:

In [ ]:
codes_weo <- weo_2016_wide$WEO.Country.Code
codes_iip <- iip$WEO.Country.Code

In [ ]:
codes_L <- setdiff(codes_weo, codes_iip)
length(codes_L)

In [ ]:
codes_R <- setdiff(codes_iip, codes_weo)
length(codes_R)

In [ ]:
codes_I <- intersect(codes_iip, codes_weo)
length(codes_I)

In [ ]:
codes_O <- union(codes_iip, codes_weo)
length(codes_O)

So in a total of 213 country codes, 5 are unique to weo, 19 are unique to iip and 189 are common across

### Left Join

First we make a "left join": We keep the country codes that weo have and do not add the codes unique to iip

In [ ]:
weo_iip_2016_L <- merge(x = weo_2016_wide,
                        y = iip,
                        by = "WEO.Country.Code",
                       all.x = T)

In [ ]:
str(weo_iip_2016_L)

12 new variables are added from iip, and number of rows is equal to that of weo_2016_wide

Now let's select those rows where codes are unique to weo:

In [ ]:
weo_iip_2016_L[weo_iip_2016_L$WEO.Country.Code %in% codes_L,]

As those rows do not appear in iip, they are shown as NA in columns imported from iip

### Right Join

We now take the codes in iip as a basis:

In [ ]:
weo_iip_2016_R <- merge(x = weo_2016_wide,
                        y = iip,
                        by = "WEO.Country.Code",
                       all.y = T)

In [ ]:
str(weo_iip_2016_R)

Number of rows is equal to that of iip

In [ ]:
weo_iip_2016_R[weo_iip_2016_R$WEO.Country.Code %in% codes_R,]

When we filter the rows for the codes unique to iip, all values in the columns inherited from weo have only NA values

### Inner Join

Now we only select those codes common to both:

In [ ]:
weo_iip_2016_I <- merge(x = weo_2016_wide,
                        y = iip,
                        by = "WEO.Country.Code",
                       all = F)

In [ ]:
str(weo_iip_2016_I)

Number of rows is equal to the number of common codes

In [ ]:
weo_iip_2016_I[weo_iip_2016_I$WEO.Country.Code %in% codes_I,]

Apart from the data missing in the original DF's, all rows have data for the weo and iip columns  

### Full Outer Join 

Now we include all codes:

In [ ]:
weo_iip_2016_O <- merge(x = weo_2016_wide,
                        y = iip,
                        by = "WEO.Country.Code",
                       all = T)

In [ ]:
str(weo_iip_2016_O)

Number of rows is equal to the length of the union set of codes from both sources

In [ ]:
weo_iip_2016_O[weo_iip_2016_O$WEO.Country.Code %in% c(codes_L, codes_R),]

When we filter for codes that are unique to either of weo or iip, we see that empty rows for both tables exist

### Join income levels

Now we will join the income level factors from weo_ppp_long_2 into weo_iip_2016_I (inner joined DF)

In [ ]:
weo_iip_2016_I2 <- merge(x = weo_iip_2016_O,
                        y = weo_ppp_long_2[weo_ppp_long_2$time == 2016,],
                        by = "WEO.Country.Code",
                       all.x = T)

In [ ]:
weo_iip_2016_I2

### Extension packages

- dplyr and data.table has advancedfunctionality for joining data objects
- sqldf package provides a unified interface to database engines as well as data frames over SQL queries. So SQL joins can be done on data objects

## Aggregating and summarizing data 

### Number of countries by year and income level

Let's first get the distribution of countries across income levels for all years using aggregate function:

In [ ]:
?aggregate

```
     aggregate(x, ...)
     
     ## Default S3 method:
     aggregate(x, ...)
     
     ## S3 method for class 'data.frame'
     aggregate(x, by, FUN, ..., simplify = TRUE, drop = TRUE)
     
     ## S3 method for class 'formula'
     aggregate(formula, data, FUN, ...,
               subset, na.action = na.omit)
     
     ## S3 method for class 'ts'
     aggregate(x, nfrequency = 1, FUN = sum, ndeltat = 1,
               ts.eps = getOption("ts.eps"), ...)
     
Arguments:

       x: an R object.

      by: a list of grouping elements, each as long as the variables in
          the data frame ‘x’.  The elements are coerced to factors
          before use.

     FUN: a function to compute the summary statistics which can be
          applied to all data subsets.

simplify: a logical indicating whether results should be simplified to
          a vector or matrix if possible.

    drop: a logical indicating whether to drop unused combinations of
          grouping values.  The non-default case ‘drop=FALSE’ has been
          amended for R 3.5.0 to drop unused combinations.

 formula: a formula, such as ‘y ~ x’ or ‘cbind(y1, y2) ~ x1 + x2’,
          where the ‘y’ variables are numeric data to be split into
          groups according to the grouping ‘x’ variables (usually
          factors).

    data: a data frame (or list) from which the variables in formula
          should be taken.
```

In [ ]:
str(weo_ppp_long_2)

In [ ]:
income_years <- aggregate(weo_ppp_long_2$Country,
                          by = as.list(weo_ppp_long_2[,c("time","income_levels")]),
                          FUN = length)

In [ ]:
str(income_years)

Now let's reshape this object into a wide format so that we see levels on columns and years on rows:

In [ ]:
income_years_wide <- reshape(income_years,
                      idvar = "time",
                      v.names = "x",
                      timevar = "income_levels",
                      direction = "wide")

In [ ]:
income_years_wide

In [ ]:
plot(x = income_years_wide$time, y = income_years_wide$x.L, col = "red", type = "l", ylim = c(0, 110))
lines(x = income_years_wide$time, income_years_wide$x.M, col = "green")
lines(x = income_years_wide$time, income_years_wide$x.H, col = "blue")

Over the last decade, while number of middle-income countries was stable, number of high-income countries increased and number of low-income countries decreased

Since LI countries cannot promote to HI countries directly, this can be interpreted as the number of countries promoted from MI to HI is more or less equal to countries promoted from LI to MI 

### Average foreign asset and foreign liabilities

In [ ]:
weo_iip_2016_I2

In [ ]:
names(weo_iip_2016_I2)

In [ ]:
str(weo_iip_2016_I2)

Now three columns will be selected:

- Assets
- Direct.investment (in assets)
- Liabilities
- Direct.investment.1 (in liabilities)
- Net.international.investment.position

and they will be divided by GDP in USD (NGDPD) and the mean values for each income level will be found:

In [ ]:
weo_iip_2016_I3 <- weo_iip_2016_I2[,c("Assets", "Direct.investment", "Liabilities",
                                      "Direct.investment.1",
                                      "Net.international.investment.position")] /
                                        weo_iip_2016_I2$NGDPD / 1000 * 100

iip_level <- aggregate(weo_iip_2016_I3,
                          by = list(weo_iip_2016_I2$income_levels),
                          FUN = function(x) round(mean(x, na.rm = T), 2))

In [ ]:
iip_level

In low income countries net international investment position to GDP rations are highly negative and only a small portion of international assets is direct investments abroad. Total international investment openness is to a limited degree

In high income countries, international investment position to GDP ratio is close to zero, however openness is extremely high: international assets is 7 times the GDP. Nearly half of international assets is of direct investment form and considering the sizes of their economies, the direct investment as assets and liabilities of high income countries must match each other (so they mostly invest in other high income countries)